# Projetar população municipal para o próximo ano

## Carregar massa de dados

In [1]:
import pandas as pd

In [2]:
dados = pd.read_csv('estimativa_dou_2014-2021.csv', sep=';',
                    dtype={'cod1': 'object', 'cod2': 'object', 'habitantes': 'int'})
dados.head()

,ano,uf,cod1,cod2,nome,habitantes
0,2014,RO,11,00015,Alta Floresta D'Oeste,25652
1,2014,RO,11,00379,Alto Alegre dos Parecis,13884
2,2014,RO,11,00403,Alto Paraíso,19841
3,2014,RO,11,00346,Alvorada D'Oeste,17228
4,2014,RO,11,00023,Ariquemes,102860


In [3]:
dados.tail()

,ano,uf,cod1,cod2,nome,habitantes
44555,2021,GO,52,22005,Vianópolis,14088
44556,2021,GO,52,22054,Vicentinópolis,9002
44557,2021,GO,52,22203,Vila Boa,6451
44558,2021,GO,52,22302,Vila Propício,5941
44559,2021,DF,53,00108,Brasília,3094325


In [4]:
dados['codigo'] = dados['cod1'] + dados['cod2']
dados.head()

,ano,uf,cod1,cod2,nome,habitantes,codigo
0,2014,RO,11,00015,Alta Floresta D'Oeste,25652,1100015
1,2014,RO,11,00379,Alto Alegre dos Parecis,13884,1100379
2,2014,RO,11,00403,Alto Paraíso,19841,1100403
3,2014,RO,11,00346,Alvorada D'Oeste,17228,1100346
4,2014,RO,11,00023,Ariquemes,102860,1100023


In [5]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44560 entries, 0 to 44559
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ano         44560 non-null  int64 
 1   uf          44560 non-null  object
 2   cod1        44560 non-null  object
 3   cod2        44560 non-null  object
 4   nome        44560 non-null  object
 5   habitantes  44560 non-null  int64 
 6   codigo      44560 non-null  object
dtypes: int64(2), object(5)
memory usage: 2.4+ MB


## Verificar maior ano e próximo ano

In [6]:
maior_ano = max(dados['ano'])
prox_ano = maior_ano + 1

print("maior ano:", maior_ano)
print("próximo ano:", prox_ano)

maior ano: 2021
próximo ano: 2022


In [7]:
dados_prox = dados[dados['ano'] == maior_ano].copy()
dados_prox['ano'] = prox_ano
dados_prox['habitantes'] = None
dados_prox.head()

,ano,uf,cod1,cod2,nome,habitantes,codigo
38990,2022,RO,11,00015,Alta Floresta D'Oeste,None,1100015
38991,2022,RO,11,00023,Ariquemes,None,1100023
38992,2022,RO,11,00031,Cabixi,None,1100031
38993,2022,RO,11,00049,Cacoal,None,1100049
38994,2022,RO,11,00056,Cerejeiras,None,1100056


In [8]:
dados_prox2 = dados_prox.copy()
dados_prox2['ano'] = dados_prox2['ano'] + 1
dados_prox2.head()

,ano,uf,cod1,cod2,nome,habitantes,codigo
38990,2023,RO,11,00015,Alta Floresta D'Oeste,None,1100015
38991,2023,RO,11,00023,Ariquemes,None,1100023
38992,2023,RO,11,00031,Cabixi,None,1100031
38993,2023,RO,11,00049,Cacoal,None,1100049
38994,2023,RO,11,00056,Cerejeiras,None,1100056


In [9]:
dados_prox3 = dados_prox.copy()
dados_prox3['ano'] = dados_prox3['ano'] + 2
dados_prox3.head()

,ano,uf,cod1,cod2,nome,habitantes,codigo
38990,2024,RO,11,00015,Alta Floresta D'Oeste,None,1100015
38991,2024,RO,11,00023,Ariquemes,None,1100023
38992,2024,RO,11,00031,Cabixi,None,1100031
38993,2024,RO,11,00049,Cacoal,None,1100049
38994,2024,RO,11,00056,Cerejeiras,None,1100056


## Testar modelo de regressão linear para um município

In [10]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

In [11]:
dados_mun = dados[dados['codigo'] == '1100023'][['ano', 'habitantes']].copy()
dados_mun

,ano,habitantes
4,2014,102860
5574,2015,104401
11141,2016,105896
16711,2017,107345
22281,2018,106168
27851,2019,107863
33421,2020,109523
38991,2021,111148


In [12]:
X = dados_mun[['ano']].values
y = dados_mun['habitantes'].values

In [13]:
model.fit(X, y)

LinearRegression()

In [14]:
previsao = int(model.predict([[prox_ano]])[0])

print("Previsão de habitantes para %d: %d" % (prox_ano, previsao))

Previsão de habitantes para 2022: 111633


## Calcular previsão para todos os municípios

In [15]:
%%time

for index, row in dados_prox.iterrows():
    codigo = row['codigo']

    dados_mun = dados[dados['codigo'] == codigo][['ano', 'habitantes']].copy()

    X = dados_mun[['ano']].values
    y = dados_mun['habitantes'].values

    model.fit(X, y)

    habitantes = int(model.predict([[prox_ano]])[0])
    habitantes2 = int(model.predict([[prox_ano + 1]])[0])
    habitantes3 = int(model.predict([[prox_ano + 2]])[0])

    dados_prox.loc[dados_prox['codigo'] == codigo, 'habitantes'] = habitantes
    dados_prox2.loc[dados_prox2['codigo'] == codigo, 'habitantes'] = habitantes2
    dados_prox3.loc[dados_prox3['codigo'] == codigo, 'habitantes'] = habitantes3

### Próximo ano

In [16]:
dados_prox.head()

,ano,uf,cod1,cod2,nome,habitantes,codigo
38990,2022,RO,11,00015,Alta Floresta D'Oeste,21718,1100015
38991,2022,RO,11,00023,Ariquemes,111633,1100023
38992,2022,RO,11,00031,Cabixi,4766,1100031
38993,2022,RO,11,00049,Cacoal,85568,1100049
38994,2022,RO,11,00056,Cerejeiras,15576,1100056


In [17]:
dados_prox.tail()

,ano,uf,cod1,cod2,nome,habitantes,codigo
44555,2022,GO,52,22005,Vianópolis,14184,5222005
44556,2022,GO,52,22054,Vicentinópolis,9145,5222054
44557,2022,GO,52,22203,Vila Boa,6625,5222203
44558,2022,GO,52,22302,Vila Propício,6000,5222302
44559,2022,DF,53,00108,Brasília,3121378,5300108


### Próximo ano + 1

In [18]:
dados_prox2.head()

,ano,uf,cod1,cod2,nome,habitantes,codigo
38990,2023,RO,11,00015,Alta Floresta D'Oeste,21169,1100015
38991,2023,RO,11,00023,Ariquemes,112685,1100023
38992,2023,RO,11,00031,Cabixi,4539,1100031
38993,2023,RO,11,00049,Cacoal,85343,1100049
38994,2023,RO,11,00056,Cerejeiras,15234,1100056


In [19]:
dados_prox2.tail()

,ano,uf,cod1,cod2,nome,habitantes,codigo
44555,2023,GO,52,22005,Vianópolis,14289,5222005
44556,2023,GO,52,22054,Vicentinópolis,9285,5222054
44557,2023,GO,52,22203,Vila Boa,6787,5222203
44558,2023,GO,52,22302,Vila Propício,6061,5222302
44559,2023,DF,53,00108,Brasília,3150481,5300108


### Próximo ano + 2

In [20]:
dados_prox3.head()

,ano,uf,cod1,cod2,nome,habitantes,codigo
38990,2024,RO,11,00015,Alta Floresta D'Oeste,20619,1100015
38991,2024,RO,11,00023,Ariquemes,113737,1100023
38992,2024,RO,11,00031,Cabixi,4312,1100031
38993,2024,RO,11,00049,Cacoal,85118,1100049
38994,2024,RO,11,00056,Cerejeiras,14891,1100056


In [21]:
dados_prox3.tail()

,ano,uf,cod1,cod2,nome,habitantes,codigo
44555,2024,GO,52,22005,Vianópolis,14393,5222005
44556,2024,GO,52,22054,Vicentinópolis,9425,5222054
44557,2024,GO,52,22203,Vila Boa,6949,5222203
44558,2024,GO,52,22302,Vila Propício,6122,5222302
44559,2024,DF,53,00108,Brasília,3179585,5300108


## Gerar arquivos CSV com previsões dos municípios

In [22]:
csv = dados_prox.drop(columns=['codigo']).sort_values(['cod1', 'nome'])
csv.to_csv('estimativa_dou_' + str(prox_ano) + '.csv', sep=';', index=False, header=False)
csv.head()

,ano,uf,cod1,cod2,nome,habitantes
38990,2022,RO,11,00015,Alta Floresta D'Oeste,21718
39015,2022,RO,11,00379,Alto Alegre dos Parecis,13027
39016,2022,RO,11,00403,Alto Paraíso,22495
39014,2022,RO,11,00346,Alvorada D'Oeste,13039
38991,2022,RO,11,00023,Ariquemes,111633


In [23]:
csv2 = dados_prox2.drop(columns=['codigo']).sort_values(['cod1', 'nome'])
csv2.to_csv('estimativa_dou_' + str(prox_ano + 1) + '.csv', sep=';', index=False, header=False)
csv2.head()

,ano,uf,cod1,cod2,nome,habitantes
38990,2023,RO,11,00015,Alta Floresta D'Oeste,21169
39015,2023,RO,11,00379,Alto Alegre dos Parecis,12898
39016,2023,RO,11,00403,Alto Paraíso,22826
39014,2023,RO,11,00346,Alvorada D'Oeste,12465
38991,2023,RO,11,00023,Ariquemes,112685


In [24]:
csv3 = dados_prox3.drop(columns=['codigo']).sort_values(['cod1', 'nome'])
csv3.to_csv('estimativa_dou_' + str(prox_ano + 2) + '.csv', sep=';', index=False, header=False)
csv3.head()

,ano,uf,cod1,cod2,nome,habitantes
38990,2024,RO,11,00015,Alta Floresta D'Oeste,20619
39015,2024,RO,11,00379,Alto Alegre dos Parecis,12770
39016,2024,RO,11,00403,Alto Paraíso,23156
39014,2024,RO,11,00346,Alvorada D'Oeste,11891
38991,2024,RO,11,00023,Ariquemes,113737
